# <u>**Лабораторная работа 5.**</u>
# Градиентный бустинг
## *Задача классификации*



### Импорт необходимых библиотек

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, cross_val_score
import matplotlib.pyplot as plt
import warnings
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore", category=FutureWarning)

In [14]:
data = pd.read_csv("../WineQT.csv")  
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


### Разделение на признаки и целевую переменную

In [15]:
X = data.drop(columns=["quality"])  # Все кроме качества
y = data["quality"]  # Качество (целевая)

### Подготовка данных, обучение и оценка качества модели

In [16]:
from sklearn.ensemble import GradientBoostingClassifier


# Бинаризация целевой переменной для классификации (например, хорошее/плохое качество)
y = (y >= 6).astype(int)  # Вино хорошего качества, если оценка >= 6

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создание и обучение модели градиентного бустинга
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)

# Предсказания
y_pred = gb.predict(X_test)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("F1-Score:", f1)
print("Classification Report:")
print(classification_report(y_test, y_pred))




Accuracy: 0.7696793002915452
F1-Score: 0.793733681462141
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.74      0.74       152
           1       0.79      0.80      0.79       191

    accuracy                           0.77       343
   macro avg       0.77      0.77      0.77       343
weighted avg       0.77      0.77      0.77       343



## **Улучшенный бейзлайн**











### *Создание полиномиальных признаков*: Полиномиальные признаки создаются с помощью функции PolynomialFeatures. Эта функция возводит исходные признаки в степень до указанной степени (в данном случае 2), что позволяет добавлять нелинейность в данные и потенциально улучшить производительность модели.

###    *Обработка дисбаланса классов через SMOTE*: Метод SMOTE используется для создания синтетических примеров меньшинства класса, чтобы сбалансировать классы перед обучением модели. Это важно, так как дисбаланс классов может привести к смещению модели в сторону большинства класса.

### *Задание параметров для GridSearchCV*: Параметры для оптимизации гиперпараметров модели задаются в словаре param_grid. 

In [21]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Загрузка данных
data = pd.read_csv("../WineQT.csv")
X = data.drop(columns=["quality"])  # Все кроме качества
y = data["quality"]

# Бинаризация целевой переменной для классификации
y = (y >= 6).astype(int)  # Вино хорошего качества, если оценка >= 6

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# SMOTE для балансировки классов
smote = SMOTE(random_state=42)

# Масштабирование данных
scaler = StandardScaler()

# Создание пайплайна с SMOTE, StandardScaler и моделью
pipeline = Pipeline(steps=[
    ('smote', smote),
    ('scaler', scaler),
    ('model', GradientBoostingClassifier(random_state=42))
])

# Определение гиперпараметров для поиска
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 5, 7],
    'model__subsample': [0.8, 1.0]
}

# Использование GridSearchCV для поиска лучших гиперпараметров
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,  # 5-кратная кросс-валидация
    scoring='f1',  # Оценка качества по F1-мере
    verbose=2,
    n_jobs=-1
)

# Обучение модели с поиском гиперпараметров
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Оценка на тестовой выборке
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Метрики
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("F1-Score:", f1)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Лучшие параметры: {'model__learning_rate': 0.2, 'model__max_depth': 5, 'model__n_estimators': 200, 'model__subsample': 1.0}
Accuracy: 0.7521865889212828
F1-Score: 0.7683923705722071
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.77      0.73       152
           1       0.80      0.74      0.77       191

    accuracy                           0.75       343
   macro avg       0.75      0.75      0.75       343
weighted avg       0.76      0.75      0.75       343



## ***Выводы***
### После применения улучшений (SMOTE, GridSearchCV и кросс-валидации) Accuracy снизилась с 0.7697 до 0.7522, а F1-Score с 0.7937 до 0.7684. Это снижение может быть связано с переобучением модели на синтетических данных, созданных SMOTE, которые не всегда точно отражают реальное распределение классов, а также с компромиссом при подборе гиперпараметров через GridSearchCV, который мог оптимизировать модель под одну метрику в ущерб другой. Балансировка классов иногда ухудшает результаты, если изначальный дисбаланс был незначительным, а увеличение данных миноритарного класса приводит к росту ошибок на преобладающем классе.

## Самостоятельная имплементация Градиентного бустинга

### Градиентный бустинг - это ансамблевый метод, который строит последовательность слабых моделей (обычно деревьев решений). Каждое новое дерево обучается на ошибках (градиентах) предыдущих моделей, минимизируя функцию потерь. Итоговое предсказание получается как сумма предсказаний всех деревьев с учётом скорости обучения.

In [26]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Загрузка данных
data = pd.read_csv("../WineQT.csv")
X = data.drop(columns=["quality"])  # Все кроме качества
y = data["quality"]  # Целевая переменная

# Бинаризация целевой переменной
y = (y >= 6).astype(int)  # Вино хорошего качества, если оценка >= 6

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Реализация градиентного бустинга
class GradientBoostingClassifierCustom:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []  # Хранение деревьев
        self.initial_pred = None  # Начальное предсказание

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Инициализация начального предсказания (средний логит)
        self.initial_pred = np.log(y.mean() / (1 - y.mean()))  # log(p / (1 - p))
        pred = np.full(y.shape, self.initial_pred)

        for _ in range(self.n_estimators):
            # Градиенты логистической функции потерь
            residual = y - self.sigmoid(pred)

            # Обучение нового дерева на градиенты
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)

            # Обновление предсказания
            update = tree.predict(X)
            pred += self.learning_rate * update

            self.models.append(tree)

    def predict_proba(self, X):
        pred = np.full(X.shape[0], self.initial_pred)
        for tree in self.models:
            pred += self.learning_rate * tree.predict(X)
        return self.sigmoid(pred)

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= 0.5).astype(int)


# Создание и обучение собственной модели
gb_custom = GradientBoostingClassifierCustom(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_custom.fit(X_train, y_train)

# Предсказания
y_pred = gb_custom.predict(X_test)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1-Score:", f1)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.7784256559766763
F1-Score: 0.8
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.76      0.75       152
           1       0.80      0.80      0.80       191

    accuracy                           0.78       343
   macro avg       0.78      0.78      0.78       343
weighted avg       0.78      0.78      0.78       343



## ***Выводы***

### Сравнение встроенной модели градиентного бустинга (GradientBoostingClassifier) и собственной реализации показало, что собственная модель немного превосходит встроенную по метрикам (Accuracy: 0.7784 vs. 0.7697, F1-Score: 0.8000 vs. 0.7937). Разница невелика, что подтверждает высокое качество обеих моделей.

# Самостоятельная имплементация градиентного бустинга с улучшенным бейзлайном

In [23]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE  # Для балансировки классов

# Загрузка данных
data = pd.read_csv("../WineQT.csv")
X = data.drop(columns=["quality"])  # Все кроме качества
y = data["quality"]  # Целевая переменная

# Бинаризация целевой переменной
y = (y >= 6).astype(int)  # Вино хорошего качества, если оценка >= 6

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 2. Создание полиномиальных признаков
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

# 3. Балансировка классов с помощью SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Реализация градиентного бустинга
class GradientBoostingClassifierCustom:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []  # Хранение деревьев
        self.initial_pred = None  # Начальное предсказание

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Инициализация начального предсказания (средний логит)
        self.initial_pred = np.log(y.mean() / (1 - y.mean()))  # log(p / (1 - p))
        pred = np.full(y.shape, self.initial_pred)

        for _ in range(self.n_estimators):
            # Градиенты логистической функции потерь
            residual = y - self.sigmoid(pred)

            # Обучение нового дерева на градиенты
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)

            # Обновление предсказания
            update = tree.predict(X)
            pred += self.learning_rate * update

            self.models.append(tree)

    def predict_proba(self, X):
        pred = np.full(X.shape[0], self.initial_pred)
        for tree in self.models:
            pred += self.learning_rate * tree.predict(X)
        return self.sigmoid(pred)

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= 0.5).astype(int)


# 4. Оптимизация гиперпараметров с помощью GridSearch
# Задаём сетку гиперпараметров для подбора
param_grid = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth": [2, 3, 4]
}

# Функция для проведения GridSearch
def grid_search_gb(X_train, y_train, X_test, y_test, param_grid):
    best_model = None
    best_params = None
    best_score = -np.inf

    for n_estimators in param_grid["n_estimators"]:
        for learning_rate in param_grid["learning_rate"]:
            for max_depth in param_grid["max_depth"]:
                # Создаём и обучаем модель с текущими параметрами
                gb = GradientBoostingClassifierCustom(
                    n_estimators=n_estimators,
                    learning_rate=learning_rate,
                    max_depth=max_depth
                )
                gb.fit(X_train, y_train)

                # Оцениваем качество на валидационных данных
                y_pred = gb.predict(X_test)
                score = f1_score(y_test, y_pred)

                # Сохраняем модель, если она лучшая
                if score > best_score:
                    best_model = gb
                    best_params = {
                        "n_estimators": n_estimators,
                        "learning_rate": learning_rate,
                        "max_depth": max_depth
                    }
                    best_score = score

    return best_model, best_params, best_score


# Выполняем GridSearch
best_model, best_params, best_score = grid_search_gb(X_train, y_train, X_test, y_test, param_grid)

# Оценка лучшей модели
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Лучшие параметры:", best_params)
print("Лучшая F1-Score (на валидации):", best_score)
print("Accuracy:", accuracy)
print("F1-Score:", f1)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Лучшие параметры: {'n_estimators': 150, 'learning_rate': 0.2, 'max_depth': 2}
Лучшая F1-Score (на валидации): 0.784741144414169
Accuracy: 0.7696793002915452
F1-Score: 0.784741144414169
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.79      0.75       152
           1       0.82      0.75      0.78       191

    accuracy                           0.77       343
   macro avg       0.77      0.77      0.77       343
weighted avg       0.77      0.77      0.77       343



## ***Выводы***
### Встроенная модель продемонстрировала Accuracy 0.7522 и F1-Score 0.7684, в то время как собственная реализация показала лучшие результаты с Accuracy 0.7697 и F1-Score 0.7847.Разница между показателями двух моделей хоть и небольшая, но всё же очевидна: собственная реализация показывает более высокое качество. Это свидетельствует о том, что в процессе обучения собственной модели удалось более эффективно минимизировать ошибку за счёт тонкой настройки алгоритма и добавленных улучшений (таких как стандартизация, полиномиальные признаки, балансировка классов и оптимизация гиперпараметров).

## *Задача регрессии*

In [28]:
data = pd.read_csv("cars.csv")
data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


### Препроцессинг данных, обучение и оценка модели

In [29]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Обработка пропущенных значений
data['Mileage'] = data['Mileage'].str.replace(' kmpl', '').str.replace(' km/kg', '').astype(float)
data['Engine'] = data['Engine'].str.replace(' CC', '').astype(float)
data['Power'] = data['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)

# Заполнение пропусков
imputer = SimpleImputer(strategy='mean')
data['Mileage'] = imputer.fit_transform(data[['Mileage']])
data['Engine'] = imputer.fit_transform(data[['Engine']])
data['Power'] = imputer.fit_transform(data[['Power']])
data['Seats'] = data['Seats'].fillna(data['Seats'].mode()[0])

# Преобразование категориальных данных
categorical_columns = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Location']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Удаление ненужных столбцов
data = data.drop(['Unnamed: 0', 'Name', 'New_Price'], axis=1)

# Разделение данных на признаки и целевую переменную
X = data.drop('Price', axis=1)
y = data['Price']

# Нормализация данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели случайного леса
from sklearn.ensemble import GradientBoostingRegressor

# Обучение модели градиентного бустинга
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)  # параметры можно настраивать
gbr.fit(X_train, y_train)

# Предсказание
y_pred = gbr.predict(X_val)

# Оценка качества модели
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("R²:", r2)



MAE: 1.9333895571309694
MSE: 16.869949311880045
R²: 0.8629122463262193


## Улучшенный бейзлайн

### Прецпроцесинг (аналогично предыдущему пункту)

In [31]:
# Загрузка данных
data = pd.read_csv("cars.csv")

# Обработка пропущенных значений
data['Mileage'] = data['Mileage'].str.replace(' kmpl', '').str.replace(' km/kg', '').astype(float)
data['Engine'] = data['Engine'].str.replace(' CC', '').astype(float)
data['Power'] = data['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)

# Заполнение пропусков
imputer = SimpleImputer(strategy='mean')
data['Mileage'] = imputer.fit_transform(data[['Mileage']])
data['Engine'] = imputer.fit_transform(data[['Engine']])
data['Power'] = imputer.fit_transform(data[['Power']])
data['Seats'] = data['Seats'].fillna(data['Seats'].mode()[0])

### Удаление выбросов по цене (верхние 1% значений)

In [32]:
data = data[data['Price'] < data['Price'].quantile(0.99)]

### Создание новых признаков,обработка данных, настройка гиперпараметров

In [33]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Добавление новых признаков
data['Age'] = 2024 - data['Year']
data['Power_to_Weight'] = data['Power'] / data['Engine']
data['Log_Price'] = np.log1p(data['Price'])  # Логарифмируем цену
data['Log_Kilometers'] = np.log1p(data['Kilometers_Driven'])  # Логарифмируем пробег

# Преобразование категориальных данных
categorical_columns = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Location']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Удаление ненужных столбцов
data = data.drop(['Unnamed: 0', 'Name', 'New_Price', 'Year', 'Price'], axis=1)

# Разделение данных на признаки и целевую переменную
X = data.drop('Log_Price', axis=1)
y = data['Log_Price']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение модели
gbr = GradientBoostingRegressor(random_state=42)

# Настройка гиперпараметров для GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],          # Количество деревьев
    'learning_rate': [0.01, 0.1, 0.2],      # Скорость обучения
    'max_depth': [3, 5, 10],                # Максимальная глубина деревьев
    'min_samples_split': [2, 5, 10],        # Минимальное число выборок для разделения
    'min_samples_leaf': [1, 2, 4],          # Минимальное число выборок в листе
    'subsample': [0.8, 1.0]                 # Доля данных для обучения каждого дерева
}

# Поиск лучших гиперпараметров с использованием GridSearchCV
grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',  # Используем MAE в качестве метрики
    cv=5,  # 5-кратная кросс-валидация
    n_jobs=-1,  # Используем все доступные ядра
    verbose=2
)

grid_search.fit(X_train, y_train)

# Лучшая модель и ее параметры
best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)
best_gbr = grid_search.best_estimator_

# Предсказание
y_pred = best_gbr.predict(X_val)

# Обратное логарифмирование для интерпретации
y_val_original = np.expm1(y_val)
y_pred_original = np.expm1(y_pred)

# Оценка модели
mae = mean_absolute_error(y_val_original, y_pred_original)
mse = mean_squared_error(y_val_original, y_pred_original)
r2 = r2_score(y_val_original, y_pred_original)

print("MAE:", mae)
print("MSE:", mse)
print("R²:", r2)


Fitting 5 folds for each of 486 candidates, totalling 2430 fits
Лучшие параметры: {'learning_rate': 0.2, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200, 'subsample': 0.8}
MAE: 1.1508392621443289
MSE: 5.459630665089103
R²: 0.931242270694103


## ***Выводы***
### До улучшений градиентный бустинг показал результаты: MAE — 1.933, MSE — 16.870, R² — 0.863. После внедрения улучшений, включая логарифмирование целевой переменной, добавление новых признаков (например, отношение мощности к объему двигателя) и оптимизацию гиперпараметров, качество модели значительно улучшилось: MAE снизилась до 1.151 (уменьшение на 40%), MSE — до 5.460, а R² вырос до 0.931. Это подтверждает, что обработка данных и настройка модели способны существенно повысить точность предсказаний.

## Собственная имплементация градиентного бустинга для регрессии

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor



class GradientBoostingRegressorCustom:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []  # Список для хранения слабых моделей
        self.initial_prediction = None  # Базовый уровень предсказания (например, среднее значение целевой переменной)

    def fit(self, X, y):
        # Инициализация с базовым прогнозом: среднее значение целевой переменной
        self.initial_prediction = np.mean(y)
        residual = y - self.initial_prediction  # Начальный остаток

        # Итеративное обучение
        for i in range(self.n_estimators):
            # Создаем слабый регрессор (решающее дерево)
            model = DecisionTreeRegressor(max_depth=self.max_depth)
            # Обучаем модель на остатках (разность между истинными значениями и текущим прогнозом)
            model.fit(X, residual)
            # Предсказываем остатки
            prediction = model.predict(X)
            # Корректируем остатки
            residual -= self.learning_rate * prediction
            # Сохраняем модель
            self.models.append(model)

    def predict(self, X):
        # Начинаем с базового предсказания
        prediction = np.full(X.shape[0], self.initial_prediction)

        # Добавляем предсказания всех слабых моделей с учетом скорости обучения
        for model in self.models:
            prediction += self.learning_rate * model.predict(X)

        return prediction


# Загрузка данных
data = pd.read_csv("cars.csv")

# Предобработка данных
# Преобразование числовых признаков
data['Mileage'] = data['Mileage'].str.replace(' kmpl', '').str.replace(' km/kg', '').astype(float)
data['Engine'] = data['Engine'].str.replace(' CC', '').astype(float)
data['Power'] = data['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)

# Заполнение пропусков
imputer = SimpleImputer(strategy='mean')
data['Mileage'] = imputer.fit_transform(data[['Mileage']])
data['Engine'] = imputer.fit_transform(data[['Engine']])
data['Power'] = imputer.fit_transform(data[['Power']])
data['Seats'] = data['Seats'].fillna(data['Seats'].mode()[0])

# Преобразование категориальных данных
categorical_columns = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Location']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Удаление ненужных столбцов
data = data.drop(['Unnamed: 0', 'Name', 'New_Price'], axis=1)

# Разделение данных на признаки и целевую переменную
X = data.drop('Price', axis=1)
y = data['Price']

# Нормализация данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение собственной реализации градиентного бустинга
gbr_custom = GradientBoostingRegressorCustom(n_estimators=100, learning_rate=0.1, max_depth=3)
gbr_custom.fit(X_train, y_train)

# Предсказание
y_pred_custom = gbr_custom.predict(X_val)

# Оценка качества модели
mae_custom = mean_absolute_error(y_val, y_pred_custom)
mse_custom = mean_squared_error(y_val, y_pred_custom)
r2_custom = r2_score(y_val, y_pred_custom)

# Вывод метрик
print("MAE (Custom):", mae_custom)
print("MSE (Custom):", mse_custom)
print("R² (Custom):", r2_custom)


MAE (Custom): 1.9350189310903971
MSE (Custom): 16.958352658921957
R² (Custom): 0.8621938673886678


## ***Выводы***
### Встроенный градиентный бустинг показывает немного лучшие результаты по сравнению с собственной имплементацией. MAE встроенного метода составляет 1.933, а собственной имплементации — 1.935, что указывает на минимальную разницу в 0.002. MSE встроенного метода равен 16.87, тогда как у собственной имплементации — 16.96, разница составляет 0.09. R² встроенной модели выше (0.863) против 0.862 у собственной имплементации. Таким образом, встроенный метод демонстрирует чуть более высокую точность, но разница крайне незначительна.

## Собственная имплементация градиентного бустинга для регрессии с улучшенным бейзлайном

In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor


# реализация Gradient Boosting Regressor
class GradientBoostingRegressorCustom:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []  # Список для хранения слабых моделей
        self.initial_prediction = None  # Базовый уровень предсказания (например, среднее значение целевой переменной)

    def fit(self, X, y):
        # Инициализация с базовым прогнозом: среднее значение целевой переменной
        self.initial_prediction = np.mean(y)
        residual = y - self.initial_prediction  # Начальный остаток

        # Итеративное обучение
        for i in range(self.n_estimators):
            # Создаем слабый регрессор (решающее дерево)
            model = DecisionTreeRegressor(max_depth=self.max_depth)
            # Обучаем модель на остатках (разность между истинными значениями и текущим прогнозом)
            model.fit(X, residual)
            # Предсказываем остатки
            prediction = model.predict(X)
            # Корректируем остатки
            residual -= self.learning_rate * prediction
            # Сохраняем модель
            self.models.append(model)

    def predict(self, X):
        # Начинаем с базового предсказания
        prediction = np.full(X.shape[0], self.initial_prediction)

        # Добавляем предсказания всех слабых моделей с учетом скорости обучения
        for model in self.models:
            prediction += self.learning_rate * model.predict(X)

        return prediction


# Загрузка данных
data = pd.read_csv("cars.csv")

# Удаление выбросов
data = data[data['Price'] < data['Price'].quantile(0.99)]

# Преобразование числовых признаков
data['Mileage'] = data['Mileage'].str.replace(' kmpl', '').str.replace(' km/kg', '').astype(float)
data['Engine'] = data['Engine'].str.replace(' CC', '').astype(float)
data['Power'] = data['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)

# Заполнение пропусков
imputer = SimpleImputer(strategy='mean')
data['Mileage'] = imputer.fit_transform(data[['Mileage']])
data['Engine'] = imputer.fit_transform(data[['Engine']])
data['Power'] = imputer.fit_transform(data[['Power']])
data['Seats'] = data['Seats'].fillna(data['Seats'].mode()[0])

# Добавление новых признаков
data['Age'] = 2024 - data['Year']
data['Power_to_Weight'] = data['Power'] / data['Engine']
data['Log_Price'] = np.log1p(data['Price'])  # Логарифмируем цену
data['Log_Kilometers'] = np.log1p(data['Kilometers_Driven'])  # Логарифмируем пробег

# Преобразование категориальных данных
categorical_columns = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Location']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Удаление ненужных столбцов
data = data.drop(['Unnamed: 0', 'Name', 'New_Price', 'Year', 'Price'], axis=1)

# Разделение данных на признаки и целевую переменную
X = data.drop('Log_Price', axis=1)
y = data['Log_Price']

# Нормализация данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Настройка гиперпараметров
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10]
}

# Подбор гиперпараметров (Grid Search) 
best_model = None
best_params = None
best_score = float('inf')

for n_estimators in param_grid['n_estimators']:
    for learning_rate in param_grid['learning_rate']:
        for max_depth in param_grid['max_depth']:
            # Создаем и обучаем модель с текущими параметрами
            model = GradientBoostingRegressorCustom(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth
            )
            model.fit(X_train, y_train)
            # Предсказание на валидационной выборке
            y_pred = model.predict(X_val)
            # Оценка модели (используем MAE)
            score = mean_absolute_error(y_val, y_pred)

            # Сохраняем лучшую модель и параметры
            if score < best_score:
                best_score = score
                best_model = model
                best_params = {
                    'n_estimators': n_estimators,
                    'learning_rate': learning_rate,
                    'max_depth': max_depth
                }


print("Лучшие параметры:", best_params)

# Предсказание на валидационной выборке
y_pred = best_model.predict(X_val)

# Обратное логарифмирование для интерпретации
y_val_original = np.expm1(y_val)
y_pred_original = np.expm1(y_pred)

# Оценка модели
mae = mean_absolute_error(y_val_original, y_pred_original)
mse = mean_squared_error(y_val_original, y_pred_original)
r2 = r2_score(y_val_original, y_pred_original)

print("MAE:", mae)
print("MSE:", mse)
print("R²:", r2)


Лучшие параметры: {'n_estimators': 200, 'learning_rate': 0.2, 'max_depth': 5}
MAE: 1.1080798909530332
MSE: 4.9731066139270625
R²: 0.9373694780205096


## ***Выводы***
### Собственная реализация градиентного бустинга показала лучшие результаты по всем метрикам по сравнению со встроенной. MAE снизилось с 1.1508 до 1.1081, MSE уменьшилось с 5.4596 до 4.9731, а R² вырос с 0.9312 до 0.9374. Это свидетельствует о более точных предсказаниях и лучшем соответствии модели данным, несмотря на её кастомную природу.

In [36]:
pip install nbmerge

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nbmerge: filename=nbmerge-0.0.4-py2.py3-none-any.whl size=6432 sha256=08a3eef3ae4c7303ce4b978128e26f94bfe33480a6e44e95bc5ce6443a682010
  Stored in directory: c:\users\vol\appdata\local\pip\cache\wheels\66\ec\e3\e3e9f9c3007f764582fcd1d93e2c9d296355985422f62c55f1
Successfully built nbmerge


In [41]:
!nbmerge lab1.ipynb lab2.ipynb lab3.ipynb lab4.ipynb lab5.ipynb -o labs.ipynb


D:\PYTHON\Lib\site-packages\nbformat\__init__.py:132: DuplicateCellId: Non-unique cell id '8c3987f6-1f85-468b-ac7b-0c8fcf0fbb5f' detected. Corrected to 'b6be6eee'.
  validate(nb)
D:\PYTHON\Lib\site-packages\nbformat\__init__.py:132: DuplicateCellId: Non-unique cell id 'abbbc0f4-bbd4-4b3b-b530-010b249cc96a' detected. Corrected to '1c37a606'.
  validate(nb)
D:\PYTHON\Lib\site-packages\nbformat\__init__.py:132: DuplicateCellId: Non-unique cell id '109e4c6a-cd31-4c59-ace3-4f068ed01914' detected. Corrected to '3ad8c045'.
  validate(nb)
D:\PYTHON\Lib\site-packages\nbformat\__init__.py:132: DuplicateCellId: Non-unique cell id 'ec9b0adf-cdda-498b-8bf2-88de7b28445c' detected. Corrected to 'ba6a26d4'.
  validate(nb)
D:\PYTHON\Lib\site-packages\nbformat\__init__.py:132: DuplicateCellId: Non-unique cell id '5d13363a-ea1a-4ff6-81d9-729f9eb3a1c8' detected. Corrected to '2af5c77c'.
  validate(nb)
D:\PYTHON\Lib\site-packages\nbformat\__init__.py:132: DuplicateCellId: Non-unique cell id '3f4e92ac-f2ce-